<a href="https://colab.research.google.com/github/fowardelcac/Analisis-ethereum-address/blob/main/api_etherscan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
import matplotlib.pyplot as plt
from requests import get
from datetime import datetime

In [47]:
API_KEY = "NZXSEDDABZ6ZTB5RMRUIBDQPVE6KS6W2SY"
BASE_URL = "https://api.etherscan.io/api"
ETHER_VALUE = 10**18

In [48]:
def get_account_balance(address):
	balance_url = make_api_url("account", "balance", address, tag="latest")
	response = get(balance_url)
	data = response.json()

	value = int(data["result"]) / ETHER_VALUE
	return value

In [46]:
address = "0x73bceb1cd57c711feac4224d062b0f6ff338501e"
get_account_balance(address)

6.949533404898927

https://api.etherscan.io/api
   ?module=account
   &action=txlist
   &address=0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC
   &startblock=0
   &endblock=99999999
   &page=1
   &offset=10
   &sort=asc
   &apikey=YourApiKeyToken

In [118]:
def get_transactions(address):
  transactions_url = make_api_url("account", "txlist", address, startblock=0, endblock=99999999, page=1, offset=10000, sort="asc")
  response = get(transactions_url)
  data = response.json()["result"]
  df = pd.DataFrame(data)
  df = df.filter(['blockNumber', 'timeStamp', 'from', 'to', 'value', 'gasPrice', 'gasUsed'], axis=1)
  df['timeStamp'] = pd.to_datetime(df['timeStamp'], unit='s')
  columnas_ = ['blockNumber', 'value', 'gasPrice', 'gasUsed']
  for i in columnas_:
    df[i] = pd.to_numeric(df[i], errors='coerce')

  df['value(Eth)'] = df.value / ETHER_VALUE
  df['Gas(Eth)'] = (df.gasPrice * df.gasUsed) / ETHER_VALUE
  return df.set_index('blockNumber').drop(['value'], axis=1)

In [119]:
address = "0x73bceb1cd57c711feac4224d062b0f6ff338501e"
data = get_transactions(address)

In [120]:
data.head()

,timeStamp,from,to,gasPrice,gasUsed,value(Eth),Gas(Eth)
blockNumber,,,,,,,
7181070,2019-02-06 02:04:20,0xea3ec2a08fee18ff4798c2d4725ded433d94151d,0x73bceb1cd57c711feac4224d062b0f6ff338501e,44000000000,21000,0.05,0.000924
7204081,2019-02-10 21:23:15,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0xea3ec2a08fee18ff4798c2d4725ded433d94151d,44000000000,21000,0.03,0.000924
7208408,2019-02-11 22:07:59,0xea3ec2a08fee18ff4798c2d4725ded433d94151d,0x73bceb1cd57c711feac4224d062b0f6ff338501e,44000000000,21000,0.17,0.000924
7212501,2019-02-12 21:08:18,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0xea3ec2a08fee18ff4798c2d4725ded433d94151d,44000000000,21000,0.10,0.000924
7212604,2019-02-12 21:47:29,0x442d1da4e0802257371384bc145fb9f8302c026f,0x73bceb1cd57c711feac4224d062b0f6ff338501e,44000000000,21000,22637.00,0.000924


In [121]:
'''
https://api.etherscan.io/api
   ?module=account
   &action=tokentx
   &contractaddress=0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
   &address=0x4e83362442b8d1bec281594cea3050c8eb01311c
   &page=1
   &offset=100
   &startblock=0
   &endblock=27025780
   &sort=asc
   &apikey=YourApiKeyToken
'''

'\nhttps://api.etherscan.io/api\n   ?module=account\n   &action=tokentx\n   &contractaddress=0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2\n   &address=0x4e83362442b8d1bec281594cea3050c8eb01311c\n   &page=1\n   &offset=100\n   &startblock=0\n   &endblock=27025780\n   &sort=asc\n   &apikey=YourApiKeyToken\n'

In [192]:
def get_tokens_tx(address):
  get_tokens_tx_url = make_api_url('account', 'tokentx', address, page=1, offset=150, startblock = 0, endblock = 27025780, sort = 'dsc')

  response = get(get_tokens_tx_url)
  data = pd.DataFrame(response.json()["result"])
  df = data.filter(['blockNumber',	'timeStamp', 'from', 'to', 'contractAddress', 'value', 'tokenName', 'tokenSymbol', 'tokenDecimal'], axis=1)
  df['timeStamp'] = pd.to_datetime(df['timeStamp'], unit='s')
  columnas_ = ['blockNumber', 'value', 'tokenDecimal']
  for i in columnas_:
    df[i] = pd.to_numeric(df[i], errors='coerce')
  df.value = df.value  / (10 ** df.tokenDecimal)
  return df.set_index('blockNumber')

In [193]:
dd = get_tokens_tx(address)

In [194]:
dd

,timeStamp,from,to,contractAddress,value,tokenName,tokenSymbol,tokenDecimal
blockNumber,,,,,,,,
7233929,2019-02-18 00:18:59,0x64b4d849965d6165d3dcfb7884e62aa0d16e8da7,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0xfa7ec2343152aeeaec018f6ae59af837840e98cb,1.000000e+03,Philanthropy,PHIL,6
7328697,2019-03-08 12:39:24,0xdc76cd25977e0a5ae17155770273ad58648900d3,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x219b9040eb7d8d8c2e8e84b87ce9ac1c83071980,1.000000e+03,Huobi Airdrop HuobiAirdrop.com,HuobiAirdrop.com,18
7328702,2019-03-08 12:40:10,0xdc76cd25977e0a5ae17155770273ad58648900d3,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x219b9040eb7d8d8c2e8e84b87ce9ac1c83071980,1.000000e+03,Huobi Airdrop HuobiAirdrop.com,HuobiAirdrop.com,18
7348360,2019-03-11 14:10:17,0xa707ee695d894db6cc1e5ee3750eb5b242d3cad9,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0xba7a6156ff985a119b5a0b35b20abef5300a55e9,1.000000e+00,Superblue,SPB,0
7483964,2019-04-01 17:36:27,0x22f85775ab87e834b37df7c13469cad5b22120c9,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x5245789633b5d0ebd21e393c3d7ead22d5ad1517,1.000000e+04,Shaggy Coin,SHAG,5
...,...,...,...,...,...,...,...,...
15679641,2022-10-05 04:52:35,0xd9585bbd98dcdc19717a9f92add429b5a8fcd549,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x34748fedad669c788aabe4fb325a4587941f7b48,1.000000e+09,TOSA INU,TOS,18
15888727,2022-11-03 09:52:59,0xf88e4cfd47ab96102ecbbdd30336eb756a0f1270,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x8f8221afbb33998d8584a2b05749ba73c37a938a,1.000000e-03,Request,REQ,18
15888769,2022-11-03 10:01:23,0xf88e4cfd47ab96102ecbbdd30336eb756a0f1270,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x8f8221afbb33998d8584a2b05749ba73c37a938a,1.000000e-03,Request,REQ,18


In [168]:
'''
https://api.etherscan.io/api
   ?module=account
   &action=tokennfttx
   &contractaddress=0x06012c8cf97bead5deae237070f9587f8e7a266d
   &address=0x6975be450864c02b4613023c2152ee0743572325
   &page=1
   &offset=100
   &startblock=0
   &endblock=27025780
   &sort=asc
   &apikey=YourApiKeyToken
'''

'\nhttps://api.etherscan.io/api\n   ?module=account\n   &action=tokennfttx\n   &contractaddress=0x06012c8cf97bead5deae237070f9587f8e7a266d\n   &address=0x6975be450864c02b4613023c2152ee0743572325\n   &page=1\n   &offset=100\n   &startblock=0\n   &endblock=27025780\n   &sort=asc\n   &apikey=YourApiKeyToken\n'

In [201]:
def get_nft_tx(address):
  get_tokens_tx_url = make_api_url('account', 'tokennfttx', address, page = 1, offset=150, startblock = 0, endblock = 27025780, sort = 'dsc')

  response = get(get_tokens_tx_url)
  data = pd.DataFrame(response.json()["result"])
  df = data.filter(['blockNumber',	'timeStamp', 'from', 'to', 'contractAddress', 'tokenName', 'tokenSymbol', 'tokenDecimal'], axis=1)
  df['timeStamp'] = pd.to_datetime(df['timeStamp'], unit='s')
  columnas_ = ['blockNumber', 'tokenDecimal']
  for i in columnas_:
    df[i] = pd.to_numeric(df[i], errors='coerce')
  return df.set_index('blockNumber')

In [202]:
get_nft_tx(address)

,timeStamp,from,to,contractAddress,tokenName,tokenSymbol,tokenDecimal
blockNumber,,,,,,,
9534174,2020-02-22 16:23:27,0x7485ac6d8534691993348d51ab0f131a19fff763,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0xd498fc4289ecbcf5c9ef033de3ceddc1fc3f6cc3,HoneyBadger Mint,HBG,0
9742989,2020-03-25 21:52:59,0x7485ac6d8534691993348d51ab0f131a19fff763,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x3a3b0dbdc0f6bc77421dcd2f55cfa087b0db9aec,Cyberpop Gallery,ETH,0
14149538,2022-02-06 01:44:08,0x658306654a6762fb88a844f4580f6c5a3079c6ad,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85,Ethereum Name Service,ENS,0
14203594,2022-02-14 09:50:46,0x0000000000000000000000000000000000000000,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x876293a24441cd7e203a3b9c4c97b2344c5c127d,InvisibIe Friends,Friends,0
14203856,2022-02-14 10:51:01,0x0000000000000000000000000000000000000000,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x876293a24441cd7e203a3b9c4c97b2344c5c127d,InvisibIe Friends,Friends,0
...,...,...,...,...,...,...,...
14598453,2022-04-16 20:07:28,0x0000000000000000000000000000000000000000,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x3e3e107c7dab448c0e7c4dc9c38bb48bd924d5f0,Apple,Apple,0
14612448,2022-04-19 00:45:39,0x0000000000000000000000000000000000000000,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x15ef843be199a1abe7185ba58fd00693f780a314,BAYC Otherside Land,BAYC,0
14641028,2022-04-23 12:20:34,0x0000000000000000000000000000000000000000,0x73bceb1cd57c711feac4224d062b0f6ff338501e,0x0d5e50c1e7cfde5219c21e0e103fe4b916805652,Squids,SQUIDS,0


In [203]:
'''
https://api.etherscan.io/api
   ?module=account
   &action=tokenbalance
   &contractaddress=0x57d90b64a1a57749b0f932f1a3395792e12e7055
   &address=0xe04f27eb70e025b78871a2ad7eabe85e61212761
   &tag=latest&apikey=YourApiKeyToken
'''

'\nhttps://api.etherscan.io/api\n   ?module=account\n   &action=tokenbalance\n   &contractaddress=0x57d90b64a1a57749b0f932f1a3395792e12e7055\n   &address=0xe04f27eb70e025b78871a2ad7eabe85e61212761\n   &tag=latest&apikey=YourApiKeyToken\n'

In [40]:
def make_api_url(module, action, address, **kwargs):
	url = BASE_URL + f"?module={module}&action={action}&address={address}&apikey={API_KEY}"

	for key, value in kwargs.items():
		url += f"&{key}={value}"

	return url

In [215]:
def get_balance_erc20(address, contractaddress):
    get_erc20_balance = make_api_url('account', 'tokenbalance', address, contractaddress = contractaddress )
    response = get(get_erc20_balance)
    data = response.json()["result"]
    return data


get_balance_erc20('0x73bceb1cd57c711feac4224d062b0f6ff338501e', '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48')

'0'

In [216]:
'''
https://api.etherscan.io/api
   ?module=stats
   &action=ethprice
   &apikey=YourApiKeyToken
'''

'\nhttps://api.etherscan.io/api\n   ?module=stats\n   &action=ethprice\n   &apikey=YourApiKeyToken\n'

In [223]:
def get_eth_px():
  px_url = BASE_URL + f"?module={'stats'}&action={'ethprice'}&apikey={API_KEY}"
  response = get(px_url)
  rdo = response.json()["result"]
  return rdo['ethusd']

get_eth_px()

'1742.27'